#### data sources<br>
[lithium supply/demand](https://www.iea.org/data-and-statistics/charts/total-lithium-demand-by-sector-and-scenario-2020-2040)<br>
the source report breaks it down by EVs only, not battery storage, so went into source report for specific number--don't use figures from this notebook


### setup

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import datetime as dt

In [3]:
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

### read in data

In [6]:
df = pd.read_csv(
    "data/raw/total-lithium-demand-by-sector-and-scenario-2020-2040.csv", skiprows=3
).rename(
    columns={
        "Unnamed: 0": "year_scenario",
        "EVs and storage": "evs_storage",
        "Other sectors": "other",
        "Share of clean energy technologies": "share",
    }
)

In [7]:
df["year"] = df["year_scenario"].str[-4:]

In [8]:
df["scenario"] = df["year_scenario"].str[:3]

In [9]:
df_slim = df[["year", "scenario", "evs_storage"]]

In [10]:
df_slim.loc[0, "scenario"] = "current"

In [11]:
df_slim

,year,scenario,evs_storage
0,2020,current,21.6
1,2030,SPS,164.0
2,2040,SPS,276.1
3,2030,SDS,383.1
4,2040,SDS,1063.3


In [48]:
alt.Chart(df_slim).mark_bar().encode(
    x=alt.X("evs_storage", stack=None, axis=alt.Axis(title="")),
    y=alt.Y("year:O", axis=alt.Axis(title="")),
    color=alt.Color("scenario:N", legend=None),
    opacity=alt.condition(alt.datum.scenario == "SDS", alt.value(0.3), alt.value(1)),
    order=alt.Order("scenario", sort="ascending"),
).properties(width=650, height=200)

alt.Chart(...)

In [42]:
alt.Chart(df_slim).mark_bar().encode(
    x=alt.X("year:O", axis=alt.Axis(title="")),
    y=alt.Y("evs_storage", stack=None, axis=alt.Axis(title="")),
    color=alt.Color("scenario:N"),
    opacity=alt.condition(alt.datum.scenario == "SDS", alt.value(0.3), alt.value(1)),
    order=alt.Order("scenario", sort="ascending"),
).properties(width=650)

alt.Chart(...)

In [13]:
df.loc[0, "scenario"] = "current"

In [14]:
df

,year_scenario,evs_storage,other,share,year,scenario
0,2020,21.6,51.8,29,2020,current
1,SPS 2030,164.0,77.7,68,2030,SPS
2,SPS 2040,276.1,97.4,74,2040,SPS
3,SDS 2030,383.1,77.7,83,2030,SDS
4,SDS 2040,1063.3,97.4,92,2040,SDS


In [15]:
df_dw = df.pivot(index="year", columns="scenario", values="evs_storage").reset_index()

In [16]:
df_dw

scenario,year,SDS,SPS,current
0,2020,NaN,NaN,21.6
1,2030,383.1,164.0,NaN
2,2040,1063.3,276.1,NaN


In [19]:
# dw.create_chart("projected lithium demand", chart_type="d3-bars", data=df_slim)
chart_id = "u3eCt"
dw.add_data(chart_id=chart_id, data=df_dw)

<Response [204]>